# Natural Language Processing Assignment 

Submitted by:<br>
   2022OG04027 &emsp;    SARANSH RAWAT <br>
   2022OG04027 &emsp;    AYUSH BANSAL  <br>
   2022OG04027 &emsp;    VIKALP SUHAG

### 1- Identify a problem of interest. Briefly explain it

Problem Selected is **Sentiment Analysis** <br> 

Sentiment analysis, also known as opinion mining, is a natural language processing (NLP) technique that involves determining the sentiment or emotion expressed in a piece of text. The goal of sentiment analysis is to understand the attitudes, opinions, and emotions conveyed in written or spoken language. It is commonly used in various applications, such as social media monitoring, customer feedback analysis, and market research.

The process typically involves analyzing the text to classify it as positive, negative, or neutral. Some advanced sentiment analysis models can also detect more nuanced emotions, such as joy, anger, sadness, or surprise. Machine learning algorithms and NLP techniques are often employed to train models on labeled datasets, allowing them to generalize and accurately classify sentiments in new, unseen text data. Sentiment analysis has practical applications in business, marketing, customer service, and social media management, providing valuable insights into public opinion and user sentiment.

### 2- Explore publically available dataset or obtain from your sources. Briefly explain it

**Dataset Information**

**Source** &emsp;  &emsp; - Real data from twitter <br>
**Size** &emsp; &emsp; &emsp; -  A sample of approximately 150,000 records. <br>
**Explanation** - All the records consist of textual informtion entered by twitter user as tweet. Every record represents a different tweet and no two records are subset of same tweet. <br>
**Scope** &emsp; &emsp; &nbsp; - All the selected tweets are related to coronavirus vaccinations

### 3- Identify library for performing chosen language task in python

We have 

In [ ]:
### 4-

### 5- Carry out data preparation as needed. Identify relevant and irrelevant attributes of the problem

Relevant attributes : <br>


## <u> Import Required Packages, define paths and read tweet data </u>

In [11]:
import os
import re
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
import string
import datetime
import numpy as np
import pandas as pd
from itertools import groupby
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.svm import SVC
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

# Display all rows and columns in cells
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 1000)

# Initialize variables
sid = SentimentIntensityAnalyzer()
ps = PorterStemmer()
stop_words = stopwords.words('english')

# Define Input and Output File paths
DATASET_FILE_PATH = os.curdir + "\\vaccination_tweets.csv"

# Read data in Dataframe
def read_data(filename):
    data = pd.read_csv(filename)
    data = data[['index', 'tweet']]
    return data

tweet_data = read_data(DATASET_FILE_PATH)
tweet_data['tweet'].head() 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0                                       Oh!, hey, Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF
1    While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've evâ€¦ https://t.co/dlCHrZjkhm
2    #coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 yearsâ€¦ https://t.co/ieYlCKBr8P
3    Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born iâ€¦ https://t.co/jqgV18kch4
4         Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTechâ€¦ https://t.co/KxbSRoBEHq
Name: tweet, dtype: object

## <u> Data Cleaning </u>

##### Remove Unicode characters

In [12]:
def text_preproc(x):
    x = x.encode('ascii', 'ignore').decode()
    return x

tweet_data['tweet'] = tweet_data.tweet.apply(text_preproc)
tweet_data['tweet'].head()

0                                    Oh!, hey, Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF
1    While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev https://t.co/dlCHrZjkhm
2    #coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years https://t.co/ieYlCKBr8P
3    Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i https://t.co/jqgV18kch4
4         Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech https://t.co/KxbSRoBEHq
Name: tweet, dtype: object

##### Make text lowercase, remove text in square brackets/arrow brackets, links, user mentions, hashtags, punctuations, words containing numbers

In [13]:
re_url = r'(?:http|ftp|https)://(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) # Remove text in square brackets
    text = re.sub(re_url, '', text)    # Remove Links
    text = re.sub('<.*?>+', '', text)  # Remove text in arrow brackets
    text = re.sub('@\w+', '', text)    # Remove User Mentions
    text = re.sub(r'#\S+', '', text)  # Remove Hashtags
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove Punctuaions
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    return text

tweet_data['tweet_cleaned'] = tweet_data['tweet'].apply(clean_text)
tweet_data['tweet_cleaned'].head()

0                                                   oh hey same folks said daikon paste could treat a cytokine storm  
1    while the world has been on the wrong side of history this year hopefully the biggest vaccination effort weve ev 
2                                                                           russian vaccine is created to last  years 
3        facts are immutable senator even when youre not ethically sturdy enough to acknowledge them  you were born i 
4                                                                       explain to me again why we need a vaccine     
Name: tweet_cleaned, dtype: object

##### Remove stop words

In [14]:
tweet_data['tweet_cleaned'] = tweet_data['tweet_cleaned'].str.split() \
    .apply(lambda x: [word for word in x if word not in stop_words]) \
    .apply(lambda x: ' '.join(x))

tweet_data['tweet_cleaned'].head()

0                      oh hey folks said daikon paste could treat cytokine storm
1     world wrong side history year hopefully biggest vaccination effort weve ev
2                                             russian vaccine created last years
3    facts immutable senator even youre ethically sturdy enough acknowledge born
4                                                           explain need vaccine
Name: tweet_cleaned, dtype: object

## <u> Word Tokenizing </u>

##### Tokenize words from clean tweet to be used to extract features

In [16]:
nltk.download('punkt')

def tokns(tweet):
    tokens = nltk.word_tokenize(tweet)
    return tokens

tweet_data['tokens'] = tweet_data['tweet_cleaned'].apply(tokns)
tweet_data[['tweet_cleaned', 'tokens']].head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


,tweet_cleaned,tokens
0,oh hey folks said daikon paste could treat cytokine storm,"[oh, hey, folks, said, daikon, paste, could, treat, cytokine, storm]"
1,world wrong side history year hopefully biggest vaccination effort weve ev,"[world, wrong, side, history, year, hopefully, biggest, vaccination, effort, weve, ev]"
2,russian vaccine created last years,"[russian, vaccine, created, last, years]"
3,facts immutable senator even youre ethically sturdy enough acknowledge born,"[facts, immutable, senator, even, youre, ethically, sturdy, enough, acknowledge, born]"
4,explain need vaccine,"[explain, need, vaccine]"


## <u> Feature Extraction </u>

##### Extract features like counts of words, unique words, characters, punctuations, consecutive characters, consecutive upper characters, consicutive punctuations, stop words, hashtags, user mentions, links and few flags mentioning hashtags, user mentions, links and punctuations

In [6]:
def get_consecutive_chars(text):
    result = [(label, sum(1 for _ in group)) for label, group in groupby(text)]
    consecutive_chars = 0
    consecutive_chars_upper = 0
    consecutive_punctuations = 0
    for i in result:
        if i[1] > 1:
            if i[0] in string.punctuation:
                consecutive_punctuations += i[1]
            elif i[0].upper() == i[0]:
                consecutive_chars_upper += i[1]
            else:
                consecutive_chars += i[1]    
    return {
        'qtd_consecutive_chars' : consecutive_chars,
        'qtd_consecutive_chars_upper': consecutive_chars_upper,
        'qtd_consecutive_punctuation' : consecutive_punctuations,
    }

tweet_data['qnt_words'] = tweet_data['tweet_cleaned'].str.split().apply(lambda x : len(x))
tweet_data['qnt_unique_words'] = tweet_data['tweet_cleaned'].str.split().apply(lambda x : len(set(x)))
tweet_data['qnt_chars'] = tweet_data['tweet'].str.len()
tweet_data['qnt_hashtags'] = tweet_data['tweet'].str.findall(r'#(\w+)').apply(lambda x : len(x))
tweet_data['qnt_user_mention'] = tweet_data['tweet'].str.findall(r'@(\w+)').apply(lambda x : len(x))
#tweet_data['qnt_punctuation'] = tweet_data['tweet'].str.replace(r'[\w\s#]+', '').apply(lambda x : len(x))
tweet_data['qnt_urls'] = tweet_data['tweet'].str.findall(re_url).apply(lambda x : len(x))
tweet_data['mean_chars_words'] = tweet_data['tweet'].str.split().apply(lambda x: np.mean([len(w) for w in x]))

#tweet_data['qnt_stop_words'] = tweet_data['tweet'].str.split() \
   # .apply(lambda x: len([w for w in x if w.lower() in stop_words]))

tweet_data['contains_hashtags'] = tweet_data['tweet'].str.findall(r'#(\w+)').apply(lambda x : 0 if len(x) == 0 else 1)
tweet_data['contains_user_mention'] = tweet_data['tweet'].str.findall(r'@(\w+)').apply(lambda x : 0 if len(x) == 0 else 1)
#tweet_data['contains_punctuation'] = tweet_data['tweet'].str.replace(r'[\w\s#]+', '').apply(lambda x : 0 if len(x) == 0 else 1)
tweet_data['contains_urls'] = tweet_data['tweet'].str.findall(re_url).apply(lambda x : len(x))

df_consecutive = tweet_data['tweet'].apply(lambda x : pd.Series(get_consecutive_chars(x)))

for col in df_consecutive.columns:
    tweet_data[col] = df_consecutive[col]

tweet_data.head()

,index,tweet,tweet_cleaned,tokens,qnt_words,qnt_unique_words,qnt_chars,qnt_hashtags,qnt_user_mention,qnt_punctuation,qnt_urls,mean_chars_words,qnt_stop_words,contains_hashtags,contains_user_mention,contains_punctuation,contains_urls,qtd_consecutive_chars,qtd_consecutive_chars_upper,qtd_consecutive_punctuation
0,1,"Oh!, hey, Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF",oh hey folks said daikon paste could treat cytokine storm,"[oh, hey, folks, said, daikon, paste, could, treat, cytokine, storm]",10,10,107,1,0,8,1,6.714286,2,1,0,1,1,2,0,2
1,2,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev https://t.co/dlCHrZjkhm",world wrong side history year hopefully biggest vaccination effort weve ev,"[world, wrong, side, history, year, hopefully, biggest, vaccination, effort, weve, ev]",11,11,139,0,0,8,1,5.666667,9,0,0,1,1,12,0,2
2,3,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years https://t.co/ieYlCKBr8P,russian vaccine created last years,"[russian, vaccine, created, last, years]",5,5,139,6,0,6,1,8.333333,2,1,0,1,1,6,0,2
3,4,"Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i https://t.co/jqgV18kch4",facts immutable senator even youre ethically sturdy enough acknowledge born,"[facts, immutable, senator, even, youre, ethically, sturdy, enough, acknowledge, born]",10,10,139,0,0,11,1,6.000000,8,0,0,1,1,6,0,2
4,5,Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech https://t.co/KxbSRoBEHq,explain need vaccine,"[explain, need, vaccine]",3,3,134,2,2,7,1,8.642857,6,1,1,1,1,10,0,2


##### Remove data where tweets with 0 no. of words after data cleaning

In [7]:
total_records = len(tweet_data)
print ('No. of records: ', total_records)
tweet_data = tweet_data[tweet_data.qnt_words != 0]
print ('No. of records after removing tweets with 0 no. of words: ', len(tweet_data))
print ('Records Removed: ', total_records - len(tweet_data))

No. of records:  145025
No. of records after removing tweets with 0 no. of words:  143805
Records Removed:  1220


##### Calculate the number of different Parts of speach (Nouns, Verbs, Interjections, Adjectives, Adverbs) in a tweet


In [8]:
def pos(tokens):
    # tokens = clean_data(tweet, lemmatize= False)
    Tagged = nltk.pos_tag(tokens)
    nouns = ['NN', 'NNS', 'NNP', 'NNPS']
    verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    interjections = ['UH']
    adjectives = ['JJ', 'JJR', 'JJS']
    adverbs = ['RB', 'RBR', 'RBS']
    noun_count, verb_count, interjection_count, adjective_count, adverb_count = 0, 0, 0, 0, 0
    no_words = len(tokens)
    for i in range(0, len(Tagged)):
        if Tagged[i][1] in nouns:
            noun_count += 1
        if Tagged[i][1] in verbs:
            verb_count += 1
        if Tagged[i][1] in interjections:
            interjection_count += 1
        if Tagged[i][1] in adjectives:
            adjective_count += 1
        if Tagged[i][1] in adverbs:
            adverb_count += 1    
    return round(float(noun_count) / float(no_words),2), round(float(verb_count) / float(no_words),2), \
           round(float(interjection_count) / float(no_words),2), round(float(adjective_count) / float(no_words),2), \
           round(float(adverb_count) / float(no_words),2)

tweet_data['nouns'], tweet_data['verbs'], tweet_data['interjections'], tweet_data['adjectives'], tweet_data['adverbs'] = zip(*tweet_data['tokens'].map(pos))

tweet_data[['tweet_cleaned', 'nouns', 'verbs', 'interjections', 'adjectives', 'adverbs']].head()

,tweet_cleaned,nouns,verbs,interjections,adjectives,adverbs
0,oh hey folks said daikon paste could treat cytokine storm,0.40,0.20,0.1,0.20,0.00
1,world wrong side history year hopefully biggest vaccination effort weve ev,0.64,0.09,0.0,0.18,0.09
2,russian vaccine created last years,0.40,0.20,0.0,0.40,0.00
3,facts immutable senator even youre ethically sturdy enough acknowledge born,0.30,0.10,0.0,0.20,0.40
4,explain need vaccine,0.00,0.33,0.0,0.00,0.33


##### Count the intensifiers in a tweet

In [9]:
def intensifier_counter(tokens):
    # tweet = clean_data(tweet, lemmatize= False)
    posC, negC = 0, 0
    for index in range(len(tokens)):
        if (index < len(tokens) - 1):
            ss_in = sid.polarity_scores(tokens[index + 1])
            if (ss_in['neg'] == 1.0):
                negC += 1
            if (ss_in['pos'] == 1.0):
                posC += 1
    return posC, negC

tweet_data['positive_intensifiers'], tweet_data['negative_intensifiers'] = \
zip(*tweet_data['tokens'].map(intensifier_counter))

tweet_data[['tokens', 'positive_intensifiers', 'negative_intensifiers']].head()

,tokens,positive_intensifiers,negative_intensifiers
0,"[oh, hey, folks, said, daikon, paste, could, treat, cytokine, storm]",1,0
1,"[world, wrong, side, history, year, hopefully, biggest, vaccination, effort, weve, ev]",1,1
2,"[russian, vaccine, created, last, years]",1,0
3,"[facts, immutable, senator, even, youre, ethically, sturdy, enough, acknowledge, born]",0,0
4,"[explain, need, vaccine]",0,0


##### Finds the most common bigrams, trigrams and skipgrams(skip 1/2 grams in a tweet)

In [10]:
def skip_grams(tokens, n, k):
    skip_gram_value = 0
    # tokens = clean_data(tweet, lemmatize= False)
    a = [x for x in nltk.skipgrams(tokens, n, k)]
    for j in range(len(a)):
        for k in range(n):
            ss = sid.polarity_scores(a[j][k])
            if (ss['pos'] == 1):
                skip_gram_value += 1
            if (ss['neg'] == 1):
                skip_gram_value -= 1
    return skip_gram_value

tks = list(tweet_data['tokens'].values)
skip_bigrams_sentiment = []
skip_trigrams_sentiment = []
skip_grams_sentiment = []

for t in tks:
        skip_bigrams_sentiment.append(skip_grams(t, 2, 0))
        skip_trigrams_sentiment.append(skip_grams(t, 3, 0))
        skip_grams_sentiment.append(skip_grams(t, 2, 2))


tweet_data['bigrams'] = skip_bigrams_sentiment
tweet_data['trigrams'] = skip_trigrams_sentiment
tweet_data['skip_grams'] = skip_grams_sentiment

tweet_data[['tokens', 'bigrams', 'trigrams', 'skip_grams']]

,tokens,bigrams,trigrams,skip_grams
0,"[oh, hey, folks, said, daikon, paste, could, treat, cytokine, storm]",2,3,5
1,"[world, wrong, side, history, year, hopefully, biggest, vaccination, effort, weve, ev]",0,1,2
2,"[russian, vaccine, created, last, years]",2,3,4
3,"[facts, immutable, senator, even, youre, ethically, sturdy, enough, acknowledge, born]",0,0,0
4,"[explain, need, vaccine]",0,0,0
5,"[anyone, useful, adviceguidance, whether, covid, vaccine, safe, whilst, breastfeeding]",4,5,9
6,"[bit, sad, claim, fame, success, patriotic, competition, usa, canada, uk]",2,4,8
7,"[many, bright, days, best, winning]",5,5,11
8,"[covid, vaccine, getting]",0,0,0
9,"[states, start, getting, monday, says]",0,0,0


##### Calculate positive word counts, negative words count and the polarity flip in a tweet i.e positive to negative or negative to positive change

In [11]:
def polarityFlip_counter(tokens):
    positive = False
    negative = False
    positive_word_count, negative_word_count, flip_count = 0, 0, 0
    for words in tokens:
        ss = sid.polarity_scores(words)
        if ss['neg'] == 1.0:
            negative = True
            negative_word_count += 1
            if positive:
                flip_count += 1
                positive = False
        elif ss['pos'] == 1.0:
            positive = True
            positive_word_count += 1
            if negative:
                flip_count += 1
                negative = False
    return positive_word_count, negative_word_count, flip_count


tweet_data['positive_word_count'], tweet_data['negative_word_count'], tweet_data['flip_count'] = \
zip(*tweet_data['tokens'].map(polarityFlip_counter))

tweet_data[['tokens', 'positive_word_count', 'negative_word_count', 'flip_count']].head()

,tokens,positive_word_count,negative_word_count,flip_count
0,"[oh, hey, folks, said, daikon, paste, could, treat, cytokine, storm]",1,0,0
1,"[world, wrong, side, history, year, hopefully, biggest, vaccination, effort, weve, ev]",1,1,1
2,"[russian, vaccine, created, last, years]",1,0,0
3,"[facts, immutable, senator, even, youre, ethically, sturdy, enough, acknowledge, born]",0,0,0
4,"[explain, need, vaccine]",0,0,0


## <u> Calculate Sentiment Score of the tweet </u>

In [12]:
def get_sentiment_score(tweet):
    return round(sid.polarity_scores(tweet)['compound'], 2)

tweet_data["sentiment_score"] = tweet_data['tweet_cleaned'].apply(get_sentiment_score)
tweet_data[['tweet_cleaned', 'sentiment_score']].head()

,tweet_cleaned,sentiment_score
0,oh hey folks said daikon paste could treat cytokine storm,0.40
1,world wrong side history year hopefully biggest vaccination effort weve ev,-0.10
2,russian vaccine created last years,0.25
3,facts immutable senator even youre ethically sturdy enough acknowledge born,0.00
4,explain need vaccine,0.00


## <u> Create Labels based on Sentiment Score </u>

##### Create multi class sentiment labels based on Sentiment Score (Positive, NEgative and Neutal Sentiments)

In [13]:
def data_label(sc):
    if sc > 0:
        label = 1
    elif sc < 0:
        label = -1
    else:
        label = 0
    return label

tweet_data['label'] = tweet_data['sentiment_score'].apply(data_label)
tweet_data[['tweet_cleaned', 'sentiment_score', 'label']].head()

,tweet_cleaned,sentiment_score,label
0,oh hey folks said daikon paste could treat cytokine storm,0.40,1
1,world wrong side history year hopefully biggest vaccination effort weve ev,-0.10,-1
2,russian vaccine created last years,0.25,1
3,facts immutable senator even youre ethically sturdy enough acknowledge born,0.00,0
4,explain need vaccine,0.00,0


## <u> Multi-class Prediction using supervised learning models </u>


##### Train different supervised learning models on 70% data and predict the labels for 30% data and calculate accuracy and other matrics (precision, recall, F1-score) to check the prediction performance of the models

##### Remove text data (tweet,tweet_cleaned, tokens) and sentiment score from the data and seperate dependent and independent variables

In [14]:
cols = ['qnt_words', 'qnt_unique_words', 'qnt_chars', 'qnt_hashtags', 'qnt_user_mention',
        'qnt_punctuation', 'qnt_urls', 'mean_chars_words', 'qnt_stop_words', 'qtd_consecutive_chars',
        'qtd_consecutive_chars_upper', 'qtd_consecutive_punctuation','contains_hashtags', 'contains_user_mention',
        'contains_punctuation', 'contains_urls', 'nouns', 'verbs','interjections', 'adjectives', 'adverbs',
        'positive_word_count', 'negative_word_count', 'flip_count', 'positive_intensifiers', 'negative_intensifiers',
        'bigrams', 'trigrams', 'skip_grams']

df = tweet_data[cols]
y = tweet_data["label"]

##### Split dependent and independent variables into train data and test data (70:30 split)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=10)

print ("X_train length: ", len(X_train))
print ("X_test length: ", len(X_test))

print ("y_train length: ", len(y_train))
print ("y_test length: ", len(y_test))

X_train length:  100663
X_test length:  43142
y_train length:  100663
y_test length:  43142


## <u> Model - Support Vector Machines (SVM) </u>

In [16]:
# Train the model
svm_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
# Predict the labels
svm_predictions = svm_linear.predict(X_test)
  
# model accuracy for X_test  
accuracy = svm_linear.score(X_test, y_test)
  
# creating a confusion matrix
svm_cm = confusion_matrix(y_test, svm_predictions)


##### SVM - Accuracy, Precision, Recall, F-Score

In [17]:
print ("SVM Accuracy: ", accuracy)
print("SVM Report: ", classification_report(y_test, svm_predictions, target_names=['Negative','Neutral','Positive']))

SVM Accuracy:  0.9638866997357564
SVM Report:                precision    recall  f1-score   support

    Negative       0.95      0.82      0.88      6631
     Neutral       1.00      1.00      1.00     22216
    Positive       0.92      0.98      0.95     14295

   micro avg       0.96      0.96      0.96     43142
   macro avg       0.96      0.93      0.94     43142
weighted avg       0.96      0.96      0.96     43142



## <u> Model - Gradient Boosting Machines (GBM) </u>

In [19]:
gbm = GradientBoostingClassifier(random_state=10).fit(X_train, y_train)
gbm_predictions = gbm.predict(X_test)

# model accuracy for X_test  
accuracy = gbm.score(X_test, y_test)

# creating a confusion matrix
gbm_cm = confusion_matrix(y_test, gbm_predictions)

##### GBM - Accuracy, Precision, Recall, F-Score

In [20]:
print ("GBM Accuracy: ", accuracy)
print(classification_report(y_test, gbm_predictions, target_names=['Negative','Neutral','Positive']))

GBM Accuracy:  0.9667377497566176
GBM Report:                precision    recall  f1-score   support

    Negative       0.94      0.85      0.89      6631
     Neutral       1.00      1.00      1.00     22216
    Positive       0.93      0.97      0.95     14295

   micro avg       0.97      0.97      0.97     43142
   macro avg       0.96      0.94      0.95     43142
weighted avg       0.97      0.97      0.97     43142



## <u> Model -  eXtreme Gradient Boosting (XGB) </u>

In [21]:
xgb = XGBClassifier().fit(X_train, y_train, use_label_encoder =False)
xgb_predictions = xgb.predict(X_test)

# model accuracy for X_test  
accuracy = xgb.score(X_test, y_test)

# creating a confusion matrix
xgb_cm = confusion_matrix(y_test, xgb_predictions)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:43:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


##### XGB - Accuracy, Precision, Recall, F-Score

In [22]:
print ("XGB Accuracy: ", accuracy)
print(classification_report(y_test, xgb_predictions, target_names=['Negative','Neutral','Positive']))

XGB Accuracy:  0.9666682119512309
XGB Report:                precision    recall  f1-score   support

    Negative       0.92      0.87      0.89      6631
     Neutral       1.00      1.00      1.00     22216
    Positive       0.94      0.96      0.95     14295

   micro avg       0.97      0.97      0.97     43142
   macro avg       0.95      0.94      0.95     43142
weighted avg       0.97      0.97      0.97     43142



## <u> Model -  Random Forest (RF) </u>

In [23]:
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(random_state = 10).fit(X_train, y_train)
rf_predictions = rf.predict(X_test)

# model accuracy for X_test  
accuracy = rf.score(X_test, y_test)

# creating a confusion matrix
rf_cm = confusion_matrix(y_test, rf_predictions)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


##### RF - Accuracy, Precision, Recall, F-Score

In [24]:
print ("RF Accuracy: ", accuracy)
print(classification_report(y_test, rf_predictions, target_names=['Negative','Neutral','Positive']))

RF Accuracy:  0.9666450326827686
RF Report:                precision    recall  f1-score   support

    Negative       0.90      0.89      0.89      6631
     Neutral       1.00      1.00      1.00     22216
    Positive       0.95      0.96      0.95     14295

   micro avg       0.97      0.97      0.97     43142
   macro avg       0.95      0.95      0.95     43142
weighted avg       0.97      0.97      0.97     43142



## <u> Results </u>
##### All 4 models predicted with almost similar results. GBM has the best accuracy (96.674%)

<img src="models_performance.png">

## 7- Comment on the solution